In [1]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from src.valuation.multiples import per_valuation
from src.valuation.graham import graham_number
from src.valuation.dcf import dcf_intrinsic_value
from src.valuation.hybrid import weighted_fair_value
from src.metrics.margin_of_safety import margin_of_safety
from src.data.fetch_price import get_latest_price
from src.data.load_fundamental import load_fundamental

In [3]:
tickers = ["AAPL", "MSFT", "GOOGL"]

In [12]:
report = {"Ticker": [], "Market Price": [], "DCF Fair Value": [], "Graham Fair Value": [], "PER Fair Value": [],"Overall Fair Value": [], 
            "DCF MoS": [], "Graham MoS": [],  "PER MoS": [], "Overall MoS": []}
for ticker in tickers:
    df = load_fundamental(project_root / f"data/raw/{ticker}.csv")

    latest = df.sort_values("year").iloc[-1]

    market_price = get_latest_price(ticker)

    fair_value_dcf = dcf_intrinsic_value(
    fcf=latest["free_cash_flow"],
    growth_rate=0.08,
    discount_rate=0.12,
    terminal_growth=0.03,
    years=5,
    shares_outstanding=latest["shares_outstanding"],
    )
    fair_value_graham = graham_number(latest["eps"], latest["book_value_per_share"])
    fair_value_per = per_valuation(latest["eps"])

    values = {
    "dcf": fair_value_dcf,
    "per": fair_value_per,
    "graham": fair_value_graham
    }

    weights = {
        "dcf": 0.5,
        "per": 0.3,
        "graham": 0.2
    }

    fair_value = weighted_fair_value(values, weights)

    mos_dcf = margin_of_safety(fair_value_dcf, market_price)
    mos_graham = margin_of_safety(fair_value_graham, market_price)
    mos_per = margin_of_safety(fair_value_per, market_price)
    mos_overall = margin_of_safety(fair_value, market_price)

    print(f"Ticker: {ticker}")
    print(f"Market Price: ${market_price:.2f}")
    print(f"DCF Fair Value: ${fair_value_dcf:.2f}, Margin of Safety: {mos_dcf:.2%}")
    print(f"Graham Fair Value: ${fair_value_graham:.2f}, Margin of Safety: {mos_graham:.2%}")
    print(f"PER Fair Value: ${fair_value_per:.2f}, Margin of Safety: {mos_per:.2%}")
    print(f"Overall Fair Value: ${fair_value:.2f}, Margin of Safety: {mos_overall:.2%}")
    print("-" * 40)
    report["Ticker"].append(ticker)
    report["Market Price"].append(np.round(market_price, 2))
    report["DCF Fair Value"].append(np.round(fair_value_dcf, 2))
    report["Graham Fair Value"].append(np.round(fair_value_graham, 2))
    report["PER Fair Value"].append(np.round(fair_value_per, 2))
    report["Overall Fair Value"].append(np.round(fair_value, 2))
    report["DCF MoS"].append('{:.2%}'.format(mos_dcf))
    report["Graham MoS"].append('{:.2%}'.format(mos_graham))
    report["PER MoS"].append('{:.2%}'.format(mos_per))
    report["Overall MoS"].append('{:.2%}'.format(mos_overall))

report_df = pd.DataFrame(report)
report_df.to_csv(project_root / f"report/fair_value_report_{datetime.now().strftime('%Y%m%d')}.csv", index=False)

Ticker: AAPL
Market Price: $278.28
DCF Fair Value: $88.83, Margin of Safety: -213.26%
Graham Fair Value: $24.21, Margin of Safety: -1049.39%
PER Fair Value: $91.95, Margin of Safety: -202.64%
Overall Fair Value: $76.84, Margin of Safety: -262.14%
----------------------------------------
Ticker: MSFT
Market Price: $478.53
DCF Fair Value: $88.83, Margin of Safety: -438.68%
Graham Fair Value: $29.55, Margin of Safety: -1519.53%
PER Fair Value: $136.95, Margin of Safety: -249.42%
Overall Fair Value: $91.41, Margin of Safety: -423.49%
----------------------------------------
Ticker: GOOGL
Market Price: $309.29
DCF Fair Value: $82.59, Margin of Safety: -274.49%
Graham Fair Value: $32.62, Margin of Safety: -848.05%
PER Fair Value: $166.95, Margin of Safety: -85.26%
Overall Fair Value: $97.90, Margin of Safety: -215.91%
----------------------------------------
